this file is just a quick try to improve the models predictions on anomalies and normal instances but even with new and different data preprocessing methods, leaving or removing many features, the models behave the same at the end, either they can precisely predict normal instances with many FN or precisely predict anomalies but then also high FP and lower TN, so either the model can predict anomalies or normal instances 

even changing the threshold of the model came to the same result.

In [1]:
# libraries imported and used throught the code
import pandas as pd
import sklearn as sklearn
import numpy as np
import sys, os
from sklearn.model_selection import train_test_split

In [2]:
# import functions
sys.path.insert(0, '../../')
from src.functions import Data, Preprocessing

dt = Data()
prep = Preprocessing()

In [3]:
# directory for  files
data_dir = os.path.join(os.getcwd(), '../../data/raw/')

# relative paths
rel_path_train_tran = 'train_transaction.csv'
rel_path_train_id = 'train_identity.csv'

# absolute paths
abs_path_train_tran = os.path.join(data_dir, rel_path_train_tran)
abs_path_train_id = os.path.join(data_dir, rel_path_train_id)

# read files
train_tran = pd.read_csv(abs_path_train_tran)
train_id = pd.read_csv(abs_path_train_id)

In [4]:
# merge  files
train = pd.merge(train_tran, train_id, on='TransactionID', how='left')

In [5]:
# pop target feature
y = train.pop('isFraud')
X = train

In [6]:
# drop columns containing 75% or more of values missing
missing_values_train = (X.isnull().sum() / len(X)) * 100
columns_to_drop_train = missing_values_train[missing_values_train >= 50].index
X = X.drop(columns=columns_to_drop_train)

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 219 entries, TransactionID to V321
dtypes: float64(207), int64(3), object(9)
memory usage: 986.7+ MB


In [8]:
# save columns
X_cols = X.columns.tolist()

In [9]:
# replace inf and -inf with NaN
X = X.replace([np.inf, -np.inf], np.nan)

# Encoding

In [10]:
X

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,0.0,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.000000,0.000000
1,2987001,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,2987002,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,2987003,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,135.0,0.000000,0.000000,0.000000,50.0,1404.0,790.0,0.000000,0.000000,0.000000
4,2987004,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,15811047,49.00,W,6550,NaN,150.0,visa,226.0,debit,...,0.0,47.950001,47.950001,47.950001,0.0,0.0,0.0,0.000000,0.000000,0.000000
590536,3577536,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
590537,3577537,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
590538,3577538,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,...,117.0,317.500000,669.500000,317.500000,0.0,2234.0,0.0,0.000000,0.000000,0.000000


In [11]:
#! pip install category_encoders
from category_encoders import BinaryEncoder

# for hig dimension data

# Binary Encoding
binary_encoder = BinaryEncoder()
X_train_binary_encoded = binary_encoder.fit_transform(X.select_dtypes(include=['object']))
# Drop the original categorical columns
X_num = X.drop(columns=X.select_dtypes(include=['object']).columns)
# Concatenate the binary-encoded DataFrame with the remaining columns
X_bin = pd.concat([X_num, X_train_binary_encoded], axis=1)

In [12]:
X_bin

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,...,M1_1,M2_0,M2_1,M3_0,M3_1,M4_0,M4_1,M4_2,M6_0,M6_1
0,2987000,86400,68.50,13926,NaN,150.0,142.0,315.0,87.0,1.0,...,1,0,1,0,1,0,0,1,0,1
1,2987001,86401,29.00,2755,404.0,150.0,102.0,325.0,87.0,1.0,...,1,1,1,1,1,0,1,0,0,1
2,2987002,86469,59.00,4663,490.0,150.0,166.0,330.0,87.0,1.0,...,1,0,1,0,1,0,1,0,1,0
3,2987003,86499,50.00,18132,567.0,150.0,117.0,476.0,87.0,2.0,...,1,1,1,1,1,0,1,0,1,0
4,2987004,86506,50.00,4497,514.0,150.0,102.0,420.0,87.0,1.0,...,1,1,1,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,15811047,49.00,6550,NaN,150.0,226.0,272.0,87.0,2.0,...,1,0,1,0,1,0,1,0,1,0
590536,3577536,15811049,39.50,10444,225.0,150.0,224.0,204.0,87.0,1.0,...,1,1,0,1,0,0,1,0,0,1
590537,3577537,15811079,30.95,12037,595.0,150.0,224.0,231.0,87.0,1.0,...,1,1,0,1,0,1,0,0,0,1
590538,3577538,15811088,117.00,7826,481.0,150.0,224.0,387.0,87.0,1.0,...,1,0,1,0,1,0,1,0,0,1


In [13]:
# missing values
from sklearn.impute import SimpleImputer
simple_imputer = SimpleImputer(strategy='mean')

In [14]:
X_simp_bin = simple_imputer.fit_transform(X_bin)

For anomaly detection algorithms like Isolation Forest, Local Outlier Factor (LOF), and One-Class SVM (OCSVM), it's generally a good practice to scale your data because these algorithms are distance-based and can be sensitive to the scale of the features.

StandardScaler is a common choice as it scales the features to have mean=0 and variance=1, which ensures that all features have the same scale. However, if you have outliers in your data, which is likely in financial fraud data, StandardScaler might not be the best choice because it's sensitive to outliers.

In this case, you might want to use a robust scaler such as RobustScaler in scikit-learn. RobustScaler scales the data according to the quantile range, making it robust to outliers.

In [15]:
# Standardize the data
from sklearn.preprocessing import RobustScaler

# Create a RobustScaler object
robust_scaler = RobustScaler()

# Fit the RobustScaler to the data and transform the data
X_simp_bin_robust = robust_scaler.fit_transform(X_simp_bin)

In [16]:
# convert the data back to dataframes
X_df = pd.DataFrame(X_simp_bin_robust, columns=X_bin.columns.tolist())

In [17]:
from sklearn.feature_selection import VarianceThreshold

# Define the selector
selector = VarianceThreshold(threshold=1)

# Determine the number of chunks
chunk_size = 10000
num_chunks = int(np.ceil(X_df.shape[0] / chunk_size))

# Initialize an empty list to hold the selected features
selected_features = []

# Process each chunk
for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, X_df.shape[0])
    X_chunk = X_df.iloc[start:end, :]

    # Fit the selector to the chunk
    X_fs = selector.fit_transform(X_chunk)

    # Get the selected features for this chunk
    chunk_selected_features = X_chunk.columns[selector.get_support(indices=True)]

    # Add the selected features to the list
    selected_features.extend(chunk_selected_features)

# Remove duplicates from the list of selected features
selected_features = list(set(selected_features))

# Print the number of features
print("Original number of features: ", X_df.shape[1])
print("Number of features after variance threshold: ", len(selected_features))
print("Number of features removed: ", X_df.shape[1] - len(selected_features))


Original number of features:  236
Number of features after variance threshold:  123
Number of features removed:  113


In [18]:
X_new = X_df[selected_features]

In [19]:
xcols = X_new.columns.tolist()

In [20]:
# convert 1 to -1 and 0 to 1
y = y.map(lambda i: -1 if i == 1 else 1)

In [21]:
# split the data into a temporary training set and a test set
X_temp, X_test, y_temp, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42, stratify=y)

# split the temporary training set into a final training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)  # 0.25 x 0.8 = 0.2

# convert arrays back to DataFrames
X_train = pd.DataFrame(X_train, columns=xcols)
X_val = pd.DataFrame(X_val, columns=xcols)
X_test = pd.DataFrame(X_test, columns=xcols)

y_train = pd.DataFrame(y_train, columns=['isFraud'])
y_val = pd.DataFrame(y_val, columns=['isFraud'])
y_test = pd.DataFrame(y_test, columns=['isFraud'])

In [22]:
from sklearn.ensemble import IsolationForest

import sys, os
import pandas as pd

In [23]:
sys.path.insert(0, '../../')
from src.functions import Data, Modeling, Evaluation

dt = Data()
mod = Modeling()
eval = Evaluation()

In [24]:
# Save column names
X_val_df = X_val

# Convert pandas dataframes to numpy arrays for memory efficiency
X_train = X_train.values
X_val = X_val.values
y_train = y_train.values
y_val = y_val.values

In [25]:
# Initialize the model
bm = IsolationForest(contamination=0.01, n_jobs=-1, random_state=42)
bm.fit(X_train)
bm_pred = bm.predict(X_val)

In [26]:
bm_cm = eval.get_cm(bm_pred, y_val)
eval.cm_inf(bm_cm, 'Base Model - Isolation Forest')

Confusion Matrix for model Base Model - Isolation Forest:

True Positives: 161
True Negatives: 112953
False Positives: 1022
False Negatives: 3972




In [27]:
# Print the classification report
eval.print_classreport(y_val, bm_pred, 'Base Model - Isolation Forest')

bm_metrics = eval.get_metrics(bm_cm, y_val, bm_pred)
eval.print_metrics(bm_metrics, 'Base Model - Isolation Forest')

Classification Report for Base Model - Isolation Forest:

              precision    recall  f1-score   support

          -1       0.14      0.04      0.06      4133
           1       0.97      0.99      0.98    113975

    accuracy                           0.96    118108
   macro avg       0.55      0.51      0.52    118108
weighted avg       0.94      0.96      0.95    118108



Metrics of Base Model - Isolation Forest:

Recall: 0.039
Precision: 0.1361
F1 Score: 0.0606
PR AUC: 0.0389
AU ROC: 0.515
Specificity: 0.991




In [29]:
from sklearn.neighbors import LocalOutlierFactor
# Initialize the model
bm = LocalOutlierFactor(n_neighbors=20, contamination=0.1, novelty=True)
bm.fit(X_train)
bm_pred = bm.predict(X_val)

In [30]:
bm_cm = eval.get_cm(bm_pred, y_val)
eval.cm_inf(bm_cm, 'Base Model - Isolation Forest')

Confusion Matrix for model Base Model - Isolation Forest:

True Positives: 590
True Negatives: 102880
False Positives: 11095
False Negatives: 3543




In [31]:
# Print the classification report
eval.print_classreport(y_val, bm_pred, 'Base Model - Isolation Forest')

bm_metrics = eval.get_metrics(bm_cm, y_val, bm_pred)
eval.print_metrics(bm_metrics, 'Base Model - Isolation Forest')

Classification Report for Base Model - Isolation Forest:

              precision    recall  f1-score   support

          -1       0.05      0.14      0.07      4133
           1       0.97      0.90      0.93    113975

    accuracy                           0.88    118108
   macro avg       0.51      0.52      0.50    118108
weighted avg       0.93      0.88      0.90    118108



Metrics of Base Model - Isolation Forest:

Recall: 0.1428
Precision: 0.0505
F1 Score: 0.0746
PR AUC: 0.0372
AU ROC: 0.5227
Specificity: 0.9027


